# Assignment

In [1]:
foo = ['Monty', 'Python']
bar = foo
foo[1] = 'Bodkin'
bar

['Monty', 'Bodkin']

The line bar = foo does not copy the contents of the variable, only its "object reference"
![assignment](http://www.nltk.org/images/array-memory.png)

In [2]:
empty = []
nested = [empty, empty, empty]
nested

[[], [], []]

In [3]:
nested[1].append('Python')
nested

[['Python'], ['Python'], ['Python']]

Observe that changing one of the items inside our nested list of lists changed them all. This is because each of the three elements is actually just a reference to one and the same list in memory.

In [4]:
nested = [[]] * 3
nested[1].append('Python')
nested[1] = ['Monty']
nested

[['Python'], ['Monty'], ['Python']]

We began with a list containing three references to a single empty list object. Then we modified that object by appending 'Python' to it, resulting in a list containing three references to a single list object ['Python']. Next, we overwrote one of those references with a reference to a new object ['Monty']. This last step modified one of the three object references inside the nested list. However, the ['Python'] object wasn't changed, and is still referenced from two places in our nested list of lists. It is crucial to appreciate this difference between modifying an object via an object reference, and overwriting an object reference.

# Equality
Python provides two ways to check that a pair of items are the same. The is operator tests for object identity. We can use it to verify our earlier observations about objects. First we create a list containing several copies of the same object, and demonstrate that they are not only identical according to ==, but also that they are one and the same object:

In [5]:
size = 5
python = ['Python']
snake_nest = [python] * size
print(snake_nest[0] == snake_nest[1] == snake_nest[2] == snake_nest[3] == snake_nest[4])
print(snake_nest[0] is snake_nest[1] is snake_nest[2] is snake_nest[3] is snake_nest[4])

True
True


In [6]:
import random
position = random.choice(range(size))
snake_nest[position] = ['Python']
print(snake_nest)
print(snake_nest[0] == snake_nest[1] == snake_nest[2] == snake_nest[3] == snake_nest[4])
print(snake_nest[0] is snake_nest[1] is snake_nest[2] is snake_nest[3] is snake_nest[4])

[['Python'], ['Python'], ['Python'], ['Python'], ['Python']]
True
False


The position'th element in the list is refering to a new 'Python'

In [8]:
[id(snake) for snake in snake_nest]

[4489933064, 4489936200, 4489933064, 4489933064, 4489933064]

# Algorithm Design

## divide-and-conquer
We attack a problem of size n by dividing it into two problems of size n/2, solve these problems, and combine their results into a solution of the original problem.

![divide-and-conqure](http://www.nltk.org/images/mergesort.png)

## Recursion
We define a function f which simplifies the problem, and calls itself to solve one or more easier instances of the same problem. It then combines the results into a solution for the original problem.

In [13]:
# iteration
def factorial1(n):
    result = 1
    for i in range(n):
        result *= (i+1)
    return result

# recursion
def factorial2(n):
    if n == 1:
        return 1
    else:
        return n*factorial2(n-1)
    
print(factorial1(10))
print(factorial2(10))
        

3628800
3628800


## dynamic programming
Dynamic programming is used when a problem contains overlapping sub-problems. Instead of computing solutions to these sub-problems repeatedly, we simply store them in a lookup table.

**Problem**: finding the number of ways of combining short and long syllables to create a meter of length n. Short syllables, marked S, take up one unit of length, while long syllables, marked L, take two. 

In [14]:
# recursion
def virahanka1(n):
    if n == 0:
        return [""]
    elif n==1:
        return ["S"]
    else:
        s = ["S" + prosody for prosody in virahanka1(n-1)]
        l = ["L" + prosody for prosody in virahanka1(n-2)]
        return s+l

#bottom-up dynamic programming
def virahanka2(n):
    lookup = [[''],['S']]
    for i in range(n-1):
        s = ['S'+prosody for prosody in lookup[i+1]]
        l = ['L'+prosody for prosody in lookup[i]]
        lookup.append(s+l)
    return lookup[n]
 
#top-down dynamic programming
def virahanka3(n,lookup={0:[""],1:["S"]}):
    if n not in lookup:
        s = ["S" + prosody for prosody in virahanka3(n-1)]
        l = ["L" + prosody for prosody in virahanka3(n-2)]
        lookup[n] = s + l
    return lookup[n]
 
# built-in memoization
from nltk import memoize
@memoize
def virahanka4(n):
    if n==0:
        return ['']
    elif n==1:
        return ['S']
    else:
        s = ["S" + prosody for prosody in virahanka4(n-1)]
        l = ["L" + prosody for prosody in virahanka4(n-2)]
        return s + l
    
print(virahanka1(4))
print(virahanka2(4))
print(virahanka3(4))
print(virahanka4(4))

['SSSS', 'SSL', 'SLS', 'LSS', 'LL']
['SSSS', 'SSL', 'SLS', 'LSS', 'LL']
['SSSS', 'SSL', 'SLS', 'LSS', 'LL']
['SSSS', 'SSL', 'SLS', 'LSS', 'LL']


The final method, in virahanka4(), is to use a Python "decorator" called memoize, which takes care of the housekeeping work done by virahanka3() without cluttering up the program. This "memoization" process stores the result of each previous call to the function along with the parameters that were used. If the function is subsequently called with the same parameters, it returns the stored result instead of recalculating it.